# Ray-tracer setup (3D)
---

In [1]:
model_name = 'model.txt'

Download ascii output of Jolien's cool Phantom model.

In [2]:
! wget https://owncloud.ster.kuleuven.be/index.php/s/6mCZjZ2erTsXq5Y/download -O $model_name

--2021-10-14 09:31:11--  https://owncloud.ster.kuleuven.be/index.php/s/6mCZjZ2erTsXq5Y/download
Resolving owncloud.ster.kuleuven.be (owncloud.ster.kuleuven.be)... 134.58.130.75
Connecting to owncloud.ster.kuleuven.be (owncloud.ster.kuleuven.be)|134.58.130.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388714637 (371M) [application/octet-stream]
Saving to: ‘model.txt’

model.txt           100%[===================>] 370.71M  10.9MB/s    in 36s     

2021-10-14 09:31:47 (10.4 MB/s) - ‘model.txt’ saved [388714637/388714637]



In [3]:
import matplotlib.pyplot as plt
import numpy             as np
import healpy            as hp
import k3d

from scipy.spatial import Delaunay, cKDTree   # Finding neighbors

In [4]:
# Read only the coordinate of the points in the file
(x,y,z, density) = np.loadtxt(model_name, skiprows=14, usecols=(0,1,2,5), unpack=True)
# Put them in a nice numpy array
points = np.array((x,y,z)).transpose()
# Convert density to cgs units (see model file)
density *= 5.941031250291510e-07
# Create a Delaunay tetrahedralisation of the points
# (We use this to find the nearest neighbors)
# https://en.wikipedia.org/wiki/Delaunay_triangulation
delaunay = Delaunay(points)

## Extracting nearset neighbours

In [4]:
# Extract Delaunay vertices (= Voronoi neighbors)
(indptr, indices) = delaunay.vertex_neighbor_vertices
neighbors = [list(indices[indptr[k]:indptr[k+1]]) for k in range(len(points))]
# neighbours is now a list of lists in which the i'th lists containts the list
# of indices of the nearest neighbours of the i'th particle.

## Defining ray directions

In [49]:
# Healpix requires the number of rays to be of the form 12*n^2
# See also https://healpy.readthedocs.io/en/latest/
nsides = 4
nrays  = 12*nsides**2
dirs   = hp.pixelfunc.pix2vec(hp.npix2nside(nrays), range(nrays))
dirs   = np.array(dirs).transpose()

## Attemt to 3D visualisation
Any tips or help to make this better very welcome!

In [46]:
# Remove last two points (they have 0 density)
density = density[:-2]
points  = points [:-2]

# Rescale to log scale to simplify plotting
ldens = np.log10(density).astype(np.float32)
minld = np.min(ldens)
maxld = np.max(ldens)

# Visualise the points
plot = k3d.plot(name='points')
plt_points = k3d.points(positions=points.astype(np.float32),
                        colors=ldens,
                        attribute=ldens,
                        opacity_function=[minld,maxld],
                        color_range=[minld,maxld],
                        shader='3dSpecular',
                        opacity=0.5)
plot += plt_points
plot.display()

Output()